## Pytorch NN API

Pytorch already includes everything we need to train a linear model in less than 10 lines of code!

In [38]:
%matplotlib inline
import random
import torch
from torch.utils import data

In [39]:
import wandb
wandb.init(project='course') # specify the project of the current run

loss,0.00011


loss,█▁▁▁▁▁▁▁▁▁


wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [40]:
true_w = torch.tensor([2, -3.4, 5, 6])
true_b = 2.4
features, labels = d2l.synthetic_data(true_w, true_b, 2000)

In [41]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 32
data_iter = load_array((features, labels), batch_size)

We can now iterate over minibtaches

In [42]:
next(iter(data_iter))

[tensor([[-0.0640, -0.2316, -1.2105,  0.9683],
         [ 1.8652, -1.8456, -1.9314, -0.2312],
         [-0.0371, -0.0543,  0.1250,  0.2585],
         [-0.0350, -1.5470, -0.3883,  1.2420],
         [ 0.0153, -1.5883,  1.8455, -0.1559],
         [-0.3330,  0.7168,  0.5310, -1.2649],
         [-0.0598,  0.0857, -1.0091, -0.3167],
         [-0.2225, -2.4045, -0.0463,  0.9433],
         [-0.0097, -0.2337, -2.0868,  1.9189],
         [ 0.0206, -0.7214,  0.7721, -1.3963],
         [-0.9205,  0.4979,  0.1712,  1.2140],
         [-0.6815,  0.5228,  0.4854, -0.9511],
         [ 1.1157,  0.2512, -1.3068,  0.9776],
         [ 1.7138, -1.3419, -0.2314, -0.3407],
         [ 0.0968,  1.1998, -1.5614,  0.5343],
         [ 0.6568,  0.7586,  0.5009, -1.3630],
         [ 0.3926, -1.0404,  0.1030,  0.7874],
         [-1.4439, -1.0687, -0.1903, -0.2318],
         [-0.1183,  0.4089,  1.0589,  2.3383],
         [ 0.2871,  0.4902, -0.3545,  1.7691],
         [ 0.1175, -1.5368, -1.4214, -1.8138],
         [-0.

For standard operations, we can **use a framework's predefined layers,**
which allow us to focus on the layers used to construct the model
rather than having to focus on the implementation.

The `Sequential` class defines a container
for several layers that will be chained together.
Given input data, a `Sequential` instance passes it through
the first layer, in turn passing the output
as the second layer's input and so forth.

The layer is said to be *fully-connected*
because each of its inputs is connected to each of its outputs
by means of a matrix-vector multiplication.

In [43]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(4, 1))

We need to initialize the model parameters. By default Pytorch initialize the weight using an uniform distribution considering the size of the layer.

You should **always** initialize your layer

<center><img src="images/weights init.jpeg" /></center>

Gradient descent doesn't move you far away from the initial starting point

The literature is full of different weight initialization techniques

You can write yours:

In [44]:
net[0].weight.data.normal_(0, 0.01) # net[0] is the first layer
net[0].bias.data.fill_(0)
net[0].weight.data

tensor([[ 0.0061, -0.0089, -0.0068, -0.0102]])

99.9999% of the time you will use one from the literature: [See Pytorch init doc](https://pytorch.org/docs/stable/nn.init.html)

I recommend **Xavier normal**. It usually works well.
If you have time/ressource, you can try different init and pick the best ;)

In [45]:
def _weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.zero_()
        
net.apply(_weights_init)
net[0].weight.data

tensor([[ 0.4055, -0.5919, -0.6559,  0.7726]])

In [46]:
net[0].bias.data

tensor([0.])

Then we need to define the loss function we will use  
The `MSELoss` class computes the mean squared error, also known as squared $L_2$ norm  
By default, it returns the average loss over examples.

In [47]:
loss = nn.MSELoss()

Weights and Biases can kee an eye on your model, login the structure and the gradient

In [48]:
wandb.watch(net, log="all", criterion=loss, log_freq=1,  log_graph=(True)) #log frequency depend on your training

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


Last piece of the puzzle, we need to define the optimizer  
When we (**instantiate an `SGD` instance,**) we will specify the parameters to optimize over (obtainable from our net via `net.parameters()`), with a dictionary of hyperparameters required by our optimization algorithm  
Minibatch stochastic gradient descent just requires that we set the value `lr`, which is set to 0.03 here.

In [49]:
optim = torch.optim.SGD(net.parameters(), lr=3e-2)

In [50]:
next(net.parameters())

Parameter containing:
tensor([[ 0.4055, -0.5919, -0.6559,  0.7726]], requires_grad=True)

Let's put everything together !

The training loop itself is strikingly similar to what we did when implementing everything from scratch.

For each minibatch, we go through the following ritual:

* Generate predictions by calling `net(X)` and calculate the loss `l` (the forward propagation).
* Calculate gradients by running the backpropagation.
* Update the model parameters by invoking our optimizer.

For good measure, we compute the loss after each epoch and print it to monitor progress.

In [51]:
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        optim.zero_grad() # please don't forget!
        l.backward() # remember: You need to tell wrt to what the gradient is computed
        optim.step() # do a step in the gradient direction
    with torch.no_grad():
        l = loss(net(features), labels) 
        print(f'epoch {epoch + 1}, loss {l.item()}')
        wandb.log({'loss': l.item()}, step=epoch)

epoch 1, loss 0.03365461900830269
epoch 2, loss 0.0001134258127422072
epoch 3, loss 9.443417366128415e-05
epoch 4, loss 9.44119892665185e-05
epoch 5, loss 9.463050810154527e-05
epoch 6, loss 9.446484909858555e-05
epoch 7, loss 9.538613085169345e-05
epoch 8, loss 9.477791900280863e-05
epoch 9, loss 9.447835327591747e-05
epoch 10, loss 9.47222433751449e-05


# ⚠️ NEVER FORGET TO ZERO_GRAD THE OPTIMIZER ⚠️

By default the optimizer accumulate the gradient!

If you don't set it back to 0, it will keep previous gradient and sum them!

If your model doesn't converge check this first!

Now let's compare the true parameters and the learned one:

In [52]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-3.1424e-04, -2.4080e-05, -2.8658e-04,  3.8433e-04])
error in estimating b: tensor([0.0006])


We can save and load model to reuse them later

In [53]:
net.state_dict()

OrderedDict([('0.weight', tensor([[ 2.0003, -3.4000,  5.0003,  5.9996]])),
             ('0.bias', tensor([2.3994]))])

In [54]:
torch.save(net.state_dict(), 'my_model.pt')

This save model's parameters as a dictionnary, but **doesn't save the structure** of the neural network

In [55]:
new_model = nn.Sequential(nn.Linear(4, 1))
new_model.load_state_dict(torch.load('my_model.pt'))
new_model.state_dict()

OrderedDict([('0.weight', tensor([[ 2.0003, -3.4000,  5.0003,  5.9996]])),
             ('0.bias', tensor([2.3994]))])